# 📄 Coletor de Notícias NSC - Santa Catarina

## 📅 Descrição

Script automatizado para coleta de notícias relacionadas a eventos climáticos em **Santa Catarina**, diretamente do portal **NDMais**. Utiliza **Selenium** com o navegador **Brave**, e faz parsing do HTML via **BeautifulSoup**.

Ideal para monitoramento de notícias sobre chuvas, enchentes, ciclones e outros eventos extremos no estado.

---

## 🛠️ Requisitos

* Python 3.x
* Bibliotecas Python:

  * `selenium`
  * `pandas`
  * `tqdm`
  * `wget`
  * `beautifulsoup4`
  * `openpyxl`

O script verifica e instala automaticamente os pacotes acima, se necessário.

---

## 🚀 Como Usar

1. Clone o repositório e execute o script principal:

```bash
jupyter notebook
```

2. Ao ser perguntado:

```bash
Deseja rodar preparação de ambiente (RECOMENDADO PARA PRIMEIRA VEZ): [sim/não]
```

Digite `sim` na primeira execução para:

* Instalar dependências
* Baixar o ChromeDriver
* Baixar o navegador Chrome (versão para automação)

3. O script iniciará a coleta de notícias automaticamente.

---

## 🔧 Funcionalidades

* Verificação e instalação automática de bibliotecas e drivers
* Download do Chrome e do ChromeDriver para automação
* Coleta de notícias por múltiplos termos e tags
* Filtragem de notícias com base em Santa Catarina
* Reinício automático do navegador em caso de falha
* Backup contínuo durante a coleta (a cada 10 páginas)
* Armazenamento dos dados em Excel, com remoção de duplicatas

---

## 📃 Estrutura dos Dados

Cada notícia coletada possui:

* `title`: Título da notícia
* `link`: URL completa
* `data`: Data de publicação
* `tag`: Palavra-chave que originou a busca (removida no Excel final)

---

## 📁 Diretórios Esperados

```plaintext
.
├── chromedriver/          # Gerado automaticamente
├── chrome/                # Chrome para automação (gerado automaticamente)
├── planilhas/
│   ├── cidade_sc1.xlsx    # Lista de municípios catarinenses (necessário)
│   └── noticias.xlsx      # Planilha gerada com as notícias filtradas
```

---

## 🧠 Lógica de Funcionamento

* As notícias são buscadas por meio de navegação automatizada por tags (termos de busca)
* A cada 10 páginas visitadas, o progresso é salvo
* A função `SCfilter()` verifica se a notícia está relacionada ao estado de SC com base em:

  * Termos como "sc", "santa catarina"
  * Nome dos municípios encontrados na planilha `cidade_sc1.xlsx`
* Notícias duplicadas são removidas antes de salvar em Excel

---

## 📈 Termos de Busca Utilizados

```python
{
  "chuvas": 52,
  "chuva em sc": 35,
  "chuvas em sc": 60,
  "chuva": 395,
  "chuva forte": 5,
  "chuvarada": 11,
  "temporal": 109,
  "tempestade": 21,
  "ciclone": 32,
  "ciclone bomba": 5,
  "ciclone extratropical": 6,
  "previsao do tempo": 708,
  "frente fria": 9,
  "enchente": 92,
  "enchentes": 22,
  "alagamento": 61,
  "alagamentos": 20,
  "deslizamento": 36,
  "deslizamentos": 9,
  "deslizamento de terra": 5
}
```

Esses termos são convertidos para o formato URL (espaços viram hifens `-`).

---

## 🧪 Ambiente Controlado

A preparação automática realiza as seguintes ações:

* Instala e atualiza bibliotecas essenciais via `pip`
* Verifica presença do Chrome e ChromeDriver e baixa automaticamente, se necessário
* Usa uma versão específica e portátil do Chrome (modo headless ativado)

---

## 🎓 Licença

Este projeto é de uso livre para fins educacionais, de pesquisa e jornalismo de dados.

---


In [ ]:
try:
    from bs4 import BeautifulSoup
    from selenium import webdriver

    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service

    import pandas as pd

    if input("Deseja rodar preparação de ambiente (RECOMENDADO PARA PRIMEIRA VEZ): [sim/não]") in ["sim", "Sim", "S", "s"]:
        raise Exception("Preparação de ambiente solicitada")
except:
    print("Configurando ambiente")
    
    import os
    import subprocess
    
    print("Checking for not installed packages...")
    
    result = subprocess.run(["pip", "list"], stdout=subprocess.PIPE, text=True)

    if not all([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Installing packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")
    
    print("All packages are installed!")
    
    
    print("Checking for outdated packages...")
    result = subprocess.run(["pip", "list", "--outdated"], stdout=subprocess.PIPE, text=True)
    
    if any([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Updating packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")

    print("All packages are updated!")
    
    import wget
    import zipfile
    
    if "chromedriver" not in os.listdir():
        print("Downloading chromedriver")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/win64/chromedriver-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chromedriver")
    else:
        print("Chromedriver found!")
    
    if "chrome" not in os.listdir():
        print("Downloading chrome")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/win64/chrome-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chrome")
    else:
        print("Chrome found!")


    from selenium import webdriver
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service

    import pandas as pd

options = webdriver.ChromeOptions()

options.binary_location = "./chrome/chrome-win64/chrome.exe"

driverpath = Service("./chromedriver/chromedriver-win64/chromedriver.exe")

options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')  # Evita problemas de memória compartilhada
options.add_argument('--disable-web-security')
options.add_argument('--disable-site-isolation-trials')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-notifications')

options.page_load_strategy = 'eager'

driver = webdriver.Chrome(service=driverpath, options=options)

def articleFormatter(article, tag): 
    return {
        "title": article.find("h3").text.strip(),
        "link": article.find("a").get_attribute_list("href")[0],
        "data": article.find("div", class_="date").text.strip(),
        "tag": tag
    }

def SCfilter(article):
    cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
    
    for key in [" sc ", "santa catarina", " sc", "sc "]:
        if key in article["title"].lower():
            return True
        
        
    for key in ["-sc-", "santa-catarina", "-sc", "sc-"]:
        if key in article["link"].split("/")[-1].lower():
            return True

    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.lower() in article["title"].lower() or cidade.lower() in article["link"].split("/")[-1].lower():
            return True

    return False

def getNewsByTags(tags):
    global driver
    allNews = []

    pageCounter = 0
    for tag in tags.keys():
        for page in range(tags[tag]):
            if pageCounter % 10 == 0:
                print(f"\nPlanilha salva com {len(allNews)} notícias para backup...")
                storeAsExcel(allNews)
                print("Salvo\n")
            
            acessed = False
            
            while not acessed:
                try:
                    driver.get(f"https://www.nsctotal.com.br/tag/{tag}?page={page}")
                    acessed = True
                except:
                    print("Erro ao acessar a página, reiniciando navegador...")
                    driver.quit()
                    driver = webdriver.Chrome(service=driverpath, options=options)
    
            try:
                WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, "date") ) )
                driver.implicitly_wait(5)
            except:
                continue            
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            news = soup.find_all('div', class_='featured-news-thumb')
            
            parsedNews = [articleFormatter(article, tag) for article in news]

            parsedNews = list(filter(SCfilter, parsedNews))
            
            allNews += parsedNews
            print(f"\nNoticias coletadas: {len(parsedNews)}\nTag: {tag}\nPágina:{page}\nTotal: {len(allNews)}\n")

            pageCounter += 1
        
        
            
    return allNews

def storeAsExcel(data):
    rows = list(map(lambda article: article.values(), data))
    df = pd.DataFrame(rows, columns=["title", "link", "data", "tag"])
    
    print(f"Número de noticias com duplicados: {len(df)}")
    
    df = df.drop("tag", axis=1)
    df = df.drop_duplicates()
    
    print(f"Número de noticias sem duplicados: {len(df)}")
    
    df.to_excel("./planilhas/noticias.xlsx", index=False)
    

searchReference = {
    "chuvas": 52,
    "chuva em sc": 35,
    "chuvas em sc": 60,
    "chuva": 395,
    "chuva forte": 5,
    "chuvarada": 11,
    "temporal": 109,
    "tempestade": 21,
    "ciclone": 32,
    "ciclone bomba": 5,
    "ciclone extratropical": 6,
    "previsao do tempo": 708,
    "frente fria": 9,
    "enchente": 92,
    "enchentes": 22,
    "alagamento": 61,
    "alagamentos": 20,
    "deslizamento": 36,
    "deslizamentos": 9,
    "deslizamento de terra": 5
}

searchReference = {tag.replace(" ", "-"): searchReference[tag] for tag in searchReference}

data = getNewsByTags(searchReference)

print(data)

storeAsExcel(data)